In [1]:
import pandas as pd
import requests
import plotly.graph_objects as go
import math
from scipy import stats
import string
import numpy as np
import time
from scipy.stats import zscore
import sys
import os
start_time = time.time()

def lineuppull(team_id, season, opp=False, ps=False):
    term = "Opponent" if opp else "Team"
    s_type = "Playoffs" if ps else "Regular Season"
    
    wowy_url = "https://api.pbpstats.com/get-wowy-stats/nba"
    print(team_id)
    wowy_params = {
        "TeamId": team_id,
        "Season": season,
        "SeasonType": s_type,
        "Type": term
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0',
    }
    
    wowy_response = requests.get(wowy_url, params=wowy_params, headers=headers)
    wowy = wowy_response.json()
    combos = wowy["multi_row_table_data"]
    frame_length = len(combos)
    df = pd.DataFrame(combos, index=[0]*frame_length)
    return df

def get_filename(team_id, year, opp=False, ps=False):
    """Generate filename based on parameters"""
    filename = f"{team_id}"
    if opp:
        filename += "_vs"
    if ps:
        filename += "_ps"
    filename += ".csv"
    return filename

def pull_onoff(years, opp=False, ps=False):
    count = 0
    if ps == False:
        player_index = pd.read_csv('index_master.csv')
    else:
         player_index = pd.read_csv('index_master_ps.csv')
    player_index = player_index[player_index.team != 'TOT']
    player_index = player_index[player_index.year > 2000]
    player_index = player_index.drop_duplicates()
    all_frames = []
    
    for year in years:
        # Create year directory if it doesn't exist
        year_dir = f"data/{year}"
        os.makedirs(year_dir, exist_ok=True)
        
        season_index = player_index[player_index.year == year].reset_index(drop=True)
        season = f"{year-1}-{str(year)[-2:]}"
        
        frames = []
        fail_list = []
        
        for team_id in season_index.team_id.unique():
            # Generate filename for this team/year combination
            filename = get_filename(team_id, year, opp, ps)
            filepath = os.path.join(year_dir, filename)
            
            # Check if file already exists
            if os.path.exists(filepath):
                print(f"File already exists for team {team_id} in {year}, skipping...")
                # Optionally read existing file and add to frames
                existing_df = pd.read_csv(filepath)
                frames.append(existing_df)
                continue
            
            try:
                df = lineuppull(team_id, season, opp=opp, ps=ps)
                df = df.reset_index(drop=True)
                df['team_id'] = team_id
                df['year'] = year
                df['season'] = season
                df['team_vs'] = opp
                
                # Save individual team file
                df.to_csv(filepath, index=False)
                time.sleep(2)
                print(f"Saved data for team {team_id} in {year}")
                
                frames.append(df)
                count += 1
                
            except Exception as e:
                print(f"Error processing team {team_id} in {year}: {str(e)}")
                fail_list.append((team_id, year))
        
        if frames:
            year_frame = pd.concat(frames)
            all_frames.append(year_frame)
            print(f'Year {year} Completed')
        
    if fail_list:
        print("\nFailed to process the following team/year combinations:")
        for team, year in fail_list:
            print(f"Team: {team}, Year: {year}")
    
    return pd.concat(all_frames) if all_frames else pd.DataFrame()
        
#pull_onoff(years,opp=True,ps=True) 
#pull_onoff(years,opp=False,ps=True) 
years=[i for i in range(2025,2026)]
df = pull_onoff(years,opp=False,ps=False) 
df = pull_onoff(years,opp=True,ps=False) 
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")
years=[i for i in range(2001,2025)]
#df = pull_onoff(years,opp=False,ps=True) 
#df = pull_onoff(years,opp=True,ps=True) 


File already exists for team 1610612738 in 2025, skipping...
File already exists for team 1610612750 in 2025, skipping...
File already exists for team 1610612743 in 2025, skipping...
File already exists for team 1610612747 in 2025, skipping...
File already exists for team 1610612742 in 2025, skipping...
File already exists for team 1610612766 in 2025, skipping...
File already exists for team 1610612737 in 2025, skipping...
File already exists for team 1610612756 in 2025, skipping...
File already exists for team 1610612749 in 2025, skipping...
File already exists for team 1610612760 in 2025, skipping...
File already exists for team 1610612746 in 2025, skipping...
File already exists for team 1610612751 in 2025, skipping...
File already exists for team 1610612765 in 2025, skipping...
File already exists for team 1610612740 in 2025, skipping...
File already exists for team 1610612739 in 2025, skipping...
File already exists for team 1610612758 in 2025, skipping...
File already exists for 

In [2]:
df

,EntityId,TeamId,Name,ShortName,RowId,TeamAbbreviation,SecondsPlayed,GamesPlayed,Minutes,PlusMinus,...,team_id,year,season,team_vs,BlockedCorner3,SecondChanceCorner3PctAssisted,BlockedLongMidRange,OffensiveGoaltends,HeaveMakes,Clear Path Fouls
0,1627759-1628369-1628401-1628436-1630202,1610612738,"Jaylen Brown, Jayson Tatum, Derrick White, Luk...","Brown, Tatum, White, Kornet, Pritchard",1627759-1628369-1628401-1628436-1630202,BOS,645.0,4,11.0,2.0,...,1610612738,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
1,1627759-1628369-1628401-1628436-201143,1610612738,"Jaylen Brown, Jayson Tatum, Derrick White, Luk...","Brown, Tatum, White, Kornet, Horford",1627759-1628369-1628401-1628436-201143,BOS,242.0,2,4.0,-11.0,...,1610612738,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
2,1627759-1628369-1628401-1628436-201950,1610612738,"Jaylen Brown, Jayson Tatum, Derrick White, Luk...","Brown, Tatum, White, Kornet, Holiday",1627759-1628369-1628401-1628436-201950,BOS,1025.0,2,17.0,-2.0,...,1610612738,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
3,1627759-1628369-1628401-1629674-1630202,1610612738,"Jaylen Brown, Jayson Tatum, Derrick White, Nee...","Brown, Tatum, White, Queta, Pritchard",1627759-1628369-1628401-1629674-1630202,BOS,682.0,2,11.0,-10.0,...,1610612738,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
4,1627759-1628369-1628401-1629674-201950,1610612738,"Jaylen Brown, Jayson Tatum, Derrick White, Nee...","Brown, Tatum, White, Queta, Holiday",1627759-1628369-1628401-1629674-201950,BOS,417.0,1,7.0,NaN,...,1610612738,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,1631117-1641707-1641718-1641729-203903,1610612762,"Walker Kessler, Taylor Hendricks, Keyonte Geor...","Kessler, Hendricks, George, Sensabaugh, Clarkson",1631117-1641707-1641718-1641729-203903,UTA,22.0,1,NaN,-1.0,...,1610612762,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
90,1631117-1641707-1641718-1642262-203903,1610612762,"Walker Kessler, Taylor Hendricks, Keyonte Geor...","Kessler, Hendricks, George, Williams, Clarkson",1631117-1641707-1641718-1642262-203903,UTA,282.0,1,5.0,-1.0,...,1610612762,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
91,1631117-1641718-1641729-1642262-203903,1610612762,"Walker Kessler, Keyonte George, Brice Sensabau...","Kessler, George, Sensabaugh, Williams, Clarkson",1631117-1641718-1641729-1642262-203903,UTA,60.0,1,1.0,2.0,...,1610612762,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
92,1631117-1641718-1642262-1642271-203903,1610612762,"Walker Kessler, Keyonte George, Cody Williams,...","Kessler, George, Williams, Filipowski, Clarkson",1631117-1641718-1642262-1642271-203903,UTA,1300.0,2,22.0,15.0,...,1610612762,2025,2024-25,True,NaN,NaN,NaN,NaN,NaN,NaN
